# 模型的保存与恢复

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

加载数据

In [2]:
mnist = input_data.read_data_sets("MNIST", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST\t10k-images-idx3-ubyte.gz
Extracting MNIST\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


设置参数batch_size的大小，计算迭代的总批次

In [3]:
batch_size = 100
n_batches = mnist.train.num_examples // batch_size

构建网络

In [4]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

In [6]:
w = tf.Variable(tf.truncated_normal([784,10], stddev=0.1))
b = tf.Variable(tf.zeros([10]) + 0.1)

预测输出

In [7]:
prediction = tf.nn.softmax(tf.matmul(x, w) + b)


定义损失函数

In [9]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))

定义优化器

In [10]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

计算正确率

In [11]:
correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

定义初始化的init

In [12]:
init = tf.global_variables_initializer()

定义保存的对象

In [13]:
saver = tf.train.Saver()

训练模型并保存

In [14]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(100):
        for batch in range(n_batches):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run([train_step], {x:batch_xs, y:batch_ys})
        saver.save(sess, save_path='saved_model/mymodel')
        acc, loss = sess.run([accuracy, cross_entropy], {x:mnist.test.images, y:mnist.test.labels})
        print("Iter: " + str(epoch) + " Loss: " + str(loss) + ", Testing Acc: " + str(acc))

Iter: 0 Loss: 1.9611119, Testing Acc: 0.6927
Iter: 1 Loss: 1.7913371, Testing Acc: 0.8105
Iter: 2 Loss: 1.7237637, Testing Acc: 0.842
Iter: 3 Loss: 1.6858625, Testing Acc: 0.8586
Iter: 4 Loss: 1.6611539, Testing Acc: 0.8721
Iter: 5 Loss: 1.6434838, Testing Acc: 0.8812
Iter: 6 Loss: 1.6301237, Testing Acc: 0.8875
Iter: 7 Loss: 1.6198373, Testing Acc: 0.8925
Iter: 8 Loss: 1.6114361, Testing Acc: 0.8961
Iter: 9 Loss: 1.6045126, Testing Acc: 0.8993
Iter: 10 Loss: 1.598698, Testing Acc: 0.9018
Iter: 11 Loss: 1.5938588, Testing Acc: 0.9037
Iter: 12 Loss: 1.5896755, Testing Acc: 0.9055
Iter: 13 Loss: 1.5861481, Testing Acc: 0.9066
Iter: 14 Loss: 1.583019, Testing Acc: 0.9078
Iter: 15 Loss: 1.5802971, Testing Acc: 0.9092
Iter: 16 Loss: 1.5779793, Testing Acc: 0.9112
Iter: 17 Loss: 1.5756525, Testing Acc: 0.9119
Iter: 18 Loss: 1.5738477, Testing Acc: 0.9121
Iter: 19 Loss: 1.5720111, Testing Acc: 0.9138
Iter: 20 Loss: 1.5704764, Testing Acc: 0.9147
Iter: 21 Loss: 1.5688945, Testing Acc: 0.9147
I

恢复模型进行对比
1. 未恢复参数的模型效果
2. 完全恢复模型参数的效果

In [17]:
with tf.Session() as sess:
    sess.run(init)
    acc, loss = sess.run([accuracy, cross_entropy], {x:mnist.test.images, y:mnist.test.labels})
    print(" Loss: " + str(loss) + ", Testing Acc: " + str(acc))

 Loss: 2.3044705, Testing Acc: 0.0996


In [18]:
with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, 'saved_model/mymodel')
    acc, loss = sess.run([accuracy, cross_entropy], {x:mnist.test.images, y:mnist.test.labels})
    print(" Loss: " + str(loss) + ", Testing Acc: " + str(acc))

INFO:tensorflow:Restoring parameters from saved_model/mymodel
 Loss: 1.5409867, Testing Acc: 0.9301
